In [ ]:
"""Given the order and rider details as described below, create a model that 
can predict rider-driven cancellation in advance 
(i.e. before getting marked as cancelled or delivered)"""

train.csv: This file contains one row for each order created in a specific time frame. The columns are:

order_id : unique id for each order
order_time: time of the creation of order by the client
order_date : date of the order
allot_time: time of allocation of order to the rider
accept_time: time of acceptance of the order by the rider (if available)
pickup_time: time of pickup of the order (if available)
delivered_time: time of delivery of the order (if available)
cancelled_time: time of cancellation of order (if the order was cancelled)
cancelled: whether the order was cancelled
rider_id: unique id for each rider
first_mile_distance: road distance from rider’s location to the pickup location
last_mile_distance: road distance from pickup location to the delivery location
allotted_orders: total number of orders allotted to the rider in the 30 days before (not including) order_date
delivered_orders: total number of orders delivered by the rider in the 30 days before (not including) order_date
undelivered_orders: total number of orders allotted to but not delivered by the rider (i.e. cancelled) in the 30 days before (not including) order_date
lifetime_order_count: total number of orders delivered by the rider at any time before order_date
reassigned_order: whether the order was reassigned to this rider
reassignment_method: if the order was reassigned, whether the reassignment was done manually (by the ops team) or automatically
reassignment_reason: a more detailed reason for the reassignment
session_time: total time the rider had been online on order_date before order_time

order time includes order date


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
ss_df = pd.read_csv('/kaggle/input/cascade-cup-22/sample_submission.csv')
ss_df.head()

In [4]:
train_df = pd.read_csv('/kaggle/input/cascade-cup-22/train.csv')
train_df.head()

In [6]:
train_df['order_time'] = pd.to_datetime(train_df['order_time'],
 format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
train_df['allot_time'] = pd.to_datetime(train_df['allot_time'],
 format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
train_df['accept_time'] = pd.to_datetime(train_df['accept_time'],
 format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
train_df['pickup_time'] = pd.to_datetime(train_df['pickup_time'],
 format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
train_df['delivered_time'] = pd.to_datetime(train_df['delivered_time'],
 format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')

In [7]:
test_df = pd.read_csv('/kaggle/input/cascade-cup-22/test.csv')
test_df.head()

In [43]:
to_be_predicted[0,:]

In [8]:
test_df['order_time'] = pd.to_datetime(test_df['order_time'],
format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
test_df['allot_time'] = pd.to_datetime(test_df['allot_time'],
format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')
test_df['accept_time'] = pd.to_datetime(test_df['accept_time'],
format = '%Y-%m-%d %H:%M:%S',errors = 'coerce')

In [11]:
print(train_df.iloc[1,:]['accept_time'])
print(train_df.iloc[0,:]['accept_time'])
print(train_df.iloc[1,:]['accept_time'] - train_df.iloc[0,:]['accept_time'])

In [12]:
train_df.columns

In [13]:
test_df.columns

In [33]:
test = test_df
test = test.drop(columns=['order_id','order_date','rider_id',
                'undelivered_orders','lifetime_order_count', 'reassignment_method','reassignment_reason', 'reassigned_order'])

In [9]:
data = train_df
#X = 
#y = pd.Series(data["target"], name="target").map({i:v for i, v in enumerate(data["target_names"])})

In [10]:
data = data.drop(columns=['pickup_time','delivered_time','cancelled_time','order_id','order_date','rider_id',
                'undelivered_orders','lifetime_order_count', 'reassignment_method','reassignment_reason', 'reassigned_order'])

In [69]:
data.columns

In [70]:
data.count()

In [11]:
data = data[data['accept_time'].notna()]

In [34]:
test = test[test['accept_time'].notna()]

In [72]:
data.count()

In [12]:
data = data.fillna(0)

In [35]:
test = test.fillna(0)

In [36]:
test.count()

In [76]:
data.count()

In [13]:
y = np.array(data['cancelled'])
y.shape

In [14]:
X = data.drop(columns=['cancelled']).to_numpy()

In [37]:
test_x = test.to_numpy()

In [38]:
to_be_predicted = test_x[:,3:]
to_be_predicted.shape

In [15]:
X = X[:,3:]
X.shape

In [80]:
X[0,:]

In [16]:
#import supervised
from sklearn.model_selection import train_test_split
#from supervised.automl import AutoML

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.11)

In [82]:
print(X_train.shape)
print(y_test.shape)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [19]:
log_reg_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors":3}, {"n_neighbors":5}]
naive_bayes_params = [{}]
svc_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}]

In [20]:
modelclasses = [["decision tree", DecisionTreeClassifier, dec_tree_params]]
#
 #   ["log regression", LogisticRegression, log_reg_params],
    
  #  ["random forest", RandomForestClassifier, rand_for_params],
   # ["k neighbors", KNeighborsClassifier, kneighbors_params],
    #["naive bayes", GaussianNB, naive_bayes_params],
    #["support vector machines", SVC, svc_params]]

In [27]:
insights = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:
        model = Model(**params)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        insights.append((modelname, model, params, score))

In [28]:
insights

In [29]:
y_pred = model.predict(X_test)

In [39]:
to_be_submitted = model.predict(to_be_predicted)

In [40]:
len(to_be_submitted)

In [41]:
(test_df['accept_time'].isna()).sum()

In [44]:

o1 = test_df[test_df['accept_time'].notna()]

In [45]:
o1.head()

In [48]:
o1_list = list(o1['order_id'])
c1_list = to_be_submitted

In [50]:
o2 = test_df[test_df['accept_time'].isna()]

In [52]:
o2_list = list(o2['order_id'])
c2_list = [1] * len(o2_list)

In [62]:
ord_id_list = o1_list+o2_list
cnc_list = c1_list+c2_list

In [61]:
c1_list = c1_list.tolist()


In [63]:
cncld_df = pd.DataFrame({'order_id':ord_id_list,'cancelled':cnc_list})
from pathlib import Path  
filepath = Path('desktop/cascade.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
cncld_df.to_csv(filepath,index=False) 


In [ ]:

automl = AutoML(total_time_limit=5*60)
automl.fit(X_train, y_train)
# Predict
y_predicted = automl.predict(X_test)

result = pd.DataFrame({"Predicted": y_predicted, "Target": np.array(y_test)})
filtro = result.Predicted == result.Target
print(filtro.value_counts(normalize=True))
# confusion matrix
df = pd.DataFrame(result)
confusion_matrix = pd.crosstab(df['Target'], df['Predicted'], rownames=['Target'], colnames=['Predicted'], margins = True)
confusion_matrix

In [7]:
train_df.count()

In [15]:
test_df.count()

In [8]:
cncld = train_df.loc[train_df['cancelled'] == 1]

In [9]:
cncld.count()

In [ ]:
train_df.iloc[389606,:]

In [11]:
cond1_tr = train_df[(train_df['order_time'] <= train_df['allot_time']) & (train_df['allot_time'] <= train_df['accept_time'])]

In [13]:
cond1_tr.count()

In [12]:
cond1_ts = test_df[(test_df['order_time'] <= test_df['allot_time']) & (test_df['allot_time'] <= test_df['accept_time'])]

In [14]:
cond1_ts.count()

In [21]:
cond2_ts = test_df[(test_df['allot_time'] > test_df['accept_time']) | (test_df['accept_time'].isna())]
cond2_ts.count()

In [ ]:
cn1 = list(ndf_test2['order_id'])

In [19]:
cond3_ts = test_df[(test_df['reassignment_method'].notnull()) & (test_df['reassignment_reason'].notnull()) &(test_df['reassigned_order'].notnull())]
cond3_ts.count()

In [ ]:
ndf_test7.head()

In [ ]:
ndf_test7.iloc[0,:]['reassignment_reason']

In [ ]:
ndf_test5 = test_df[(test_df['session_time'].isna())]

In [ ]:
ndf_test8 = test_df[(test_df['session_time'] == 0)]

In [ ]:
cn3 = list(ndf_test8['session_time'])

In [ ]:
ndf_test8.head()

In [ ]:
ndf_test6 = test_df[(test_df['undelivered_orders'].where)]

In [ ]:
ndf_test4 = test_df[(test_df['accept_time'].isna())]
cn2 = list(ndf_test4['order_id'])
cn1 = cn1 + cn2 + cn3

In [ ]:
cnd = list(set(cn1))
len(cnd)

In [ ]:
ordl = list(test_df['order_id'])

In [ ]:
order_id_list = []
cancelled_list = []
for o in ordl:
    if(o in cnd):
        order_id_list.append(o)
        cancelled_list.append(1)
    else:
        order_id_list.append(o)
        cancelled_list.append(0)

In [ ]:
cncld_df = pd.DataFrame({'order_id':order_id_list,'cancelled':cancelled_list})

In [ ]:
cncld_df = pd.DataFrame({'order_id':order_id_list,'cancelled':cancelled_list})
from pathlib import Path  
filepath = Path('desktop/cascade.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
cncld_df.to_csv(filepath,index=False) 


In [ ]:
test_df.count()

In [ ]:
print(ndf_test5.count())
#print(ndf_test2.count())

In [ ]:
ndf2_cncld = ndf2.loc[ndf2['cancelled'] == 1]

In [ ]:
ndf2_cncld.head()

In [ ]:
(ndf2_cncld['pickup_time'].isnull()).sum()

In [ ]:
(train_df['pickup_time'].isnull()).sum()

In [ ]:
#ndf2_cncld[]
ndf2_cnc_pick = ndf2_cncld[ndf2_cncld['pickup_time'].notnull()]

In [ ]:
ndf2_cnc_pick.count()

In [ ]:
ndf2_cnc_pick.head()

In [ ]:
ndf3 = train_df[(train_df['order_time'] == train_df['allot_time'])]
#& (train_df['allot_time'] < train_df['accept_time'])

In [ ]:
ndf3.head()